In [1]:
import sys
import textwrap

sys.path.append("..")

### LLM

In [2]:
from model.llm import OpenaiLLM
llm = OpenaiLLM()

### Ingestion

In [3]:
from rag import chunk
from rag import load
from rag import utils
from rag.vectordb import ChromaDB

loader_cfg = utils.read_config("config.yaml", "ragchain")["ingestion_config"]["loader"]
chunker_cfg = utils.read_config("config.yaml", "ragchain")["ingestion_config"]["chunker"]
vectordb_cfg = utils.read_config("config.yaml", "ragchain")["vectordb_config"]

document_path = "../docs/TayXueHao-Resume.pdf"
chromadb = ChromaDB(vectordb_cfg=vectordb_cfg, debug=True)
loader = load.PDFLoader(loader_cfg, debug=True)
documents = loader(doc_path=document_path)
chunker = chunk.RecursiveChunker(chunker_cfg=chunker_cfg, debug=True)
chunker(documents, chromadb)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Vector database configurations
Vectordb: Chroma
Embedding model: text-embedding-3-small
Loader:  ABCMeta
{'extract_images': False, 'file_path': '../docs/TayXueHao-Resume.pdf'}
chunker: RecursiveCharacterTextSplitter
Documents: [Document(page_content='Data Scientist\nDriven by a genuine passion in Artificial Intelligence, I bring a solid academic\nfoundation with a CGPA of 4.00. As an aspiring Data Scientist, I am eager to\nlearn and contribute my expertise in Machine Learning and Artificial\nIntelligence to your organization. With focus of creating value through cutting-\nedge technology, I specialize in the dynamic realm of Computer Vision, Data\nScience and Deep Learning. Tay Xue Hao\nPersonal Details\nNotable Projects012-462 8936\ntayxhwork@gmail.com\n16, Lorong 42, Taman Petani\nJaya, 08000 Sungai Petani,\nKedah.\nhttps://www.linkedin.com/in/t\nay-xue-hao/EmailPhone\nLinkedInAddress\nFacial Emotion Recognition using\nConvolutional Neural Network (Uni)\nAutomated Cryptocurrency Trad

In [5]:
for chunk in chunker.chunks:
    print(chunk.metadata)

{'source': '../docs/TayXueHao-Resume.pdf', 'page': 0}
{'source': '../docs/TayXueHao-Resume.pdf', 'page': 0}
{'source': '../docs/TayXueHao-Resume.pdf', 'page': 0}
{'source': '../docs/TayXueHao-Resume.pdf', 'page': 1}
{'source': '../docs/TayXueHao-Resume.pdf', 'page': 1}
{'source': '../docs/TayXueHao-Resume.pdf', 'page': 1}


### Query Documents

In [24]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
                                          <context>
                                          {context}
                                          </context>

                                          Question: {input}
                                          """)

document_chain = create_stuff_documents_chain(llm, prompt)

In [25]:
from langchain.chains import create_retrieval_chain

vectordb = chromadb.vectordb
retriever = vectordb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

- `retrieval_chain` retrieves contexts based on input
- `document_chain` feeds these input to the prompt and llm

In [26]:
response = retrieval_chain.invoke({"input": "What are Xue Hao's interests?"})
llm_response = textwrap.fill(response["answer"], width=80)
print(llm_response)

Xue Hao's interests include Reading, Gym Workout, Swimming, and Artificial
Intelligence.


Interestingly, it combines its knowledge about langsmith and the contexts given which is my resume.

### Advanced Retrieval: MultiQueryRetriever

This retriever technique accepts the user's query and generate multiple related questions and retrieves documents for all the questions. We can reuse the chains declared from above, and only change the retriever method. This is the power of Langchain's abstraction.

In [27]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=llm)

In [28]:
retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [29]:
response = retrieval_chain.invoke({"input": "What are Xue Hao's interests?"})
llm_response = textwrap.fill(response["answer"], width=80)
print(llm_response)

Xue Hao's interests include Reading, Gym Workout, Swimming, and Artificial
Intelligence.


To understand more about langchain's retrievers, visit the link below. Langchain's documentation explains the different rypes of retrieval ypes they support, when to use them, description, and whether they use an llm or not.  
- Langchain retriever documentation: https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/